In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import importlib
import os
import glob

import sys
sys.path.append('/home/rwollman/MyProjects/AH/Repos/dredFISH')
from dredFISH.Analysis.TissueGraph import *
from dredFISH.Analysis.Classification import *
from dredFISH.Visualization.Viz import *
from dredFISH.Analysis import Classification
from dredFISH.Utils import geomu, fileu

plt.rcParams['figure.facecolor'] = 'white' 

In [ ]:
pth = os.getcwd()
parts = os.path.split(pth)
parts2 = os.path.split(parts[0])
parts3 = os.path.split(parts2[0])
inputpath = os.path.join(parts[0],parts3[1])

In [ ]:
inputpath

In [ ]:
config = fileu.load_config_module(inputpath)
hybes = [c[0] for c in config.bitmap if c[0]!= 'PolyT']

In [ ]:
TMG = TissueMultiGraph(inputpath = inputpath,redo = True)
TMG.create_cell_layer(build_spatial_graph = False,build_feature_graph = False,hybes=hybes,norm='none')

In [ ]:
XY = TMG.Layers[0].XY
in_large_comp = geomu.in_graph_large_connected_components(XY,large_comp_def = 0.01,plot_comp = True,max_dist = 100)

In [ ]:
print(f"before filtering: {TMG.N[0]} cells")
TMG.Layers[0].filter(in_large_comp)
print(f"after filtering: {TMG.N[0]} cells")

In [ ]:
vmin,vmax = np.percentile(TMG.Layers[0].adata.obs['dapi'],[1,99])
vmin,vmax

In [ ]:
DapiValueDistributions(TMG,min_dapi_line = vmin,max_dapi_line=vmax,figsize = (4,4)).show()

In [ ]:
to_keep = np.logical_and(TMG.Layers[0].adata.obs['dapi']>vmin,
                         TMG.Layers[0].adata.obs['dapi']<vmax) 
print(f"before filtering: {TMG.N[0]} cells")
TMG.Layers[0].filter(to_keep)
print(f"after filtering: {TMG.N[0]} cells")

In [ ]:
data = TMG.Layers[0].adata
converter = {r:h.split('hybe')[-1] for r,h,c in config.bitmap}
''.join(config.bitmap[np.where(data.X.mean(0)==value)[0][0]][1].split('hybe')[-1]+',' for value in sorted(data.X.mean(0)))

In [ ]:
TMG.Layers[0].build_spatial_graph()

In [ ]:
data = TMG.Layers[0].adata
# Create a 5x5 grid of subplots
fig, axs = plt.subplots(5, 5, figsize=(25, 25))
x = data.obs['stage_x']
y = data.obs['stage_y']
# Flatten the axs array so that we can iterate over it using a single loop
axs = axs.ravel()
num_plots = data.X.shape[1]
# Iterate over the subplots and plot each scatter plot
for i in range(num_plots):
    c = data.X[:,i].copy()
    vmin,vmax = np.percentile(c[c!=0],[5,95])
    ax = axs[i]
    scatter_plot = ax.scatter(x, y,c=c,vmin=vmin,vmax=vmax,s=0.1,marker='x',cmap='jet')  # Adjust 's' for marker size if needed
    fig.colorbar(scatter_plot, ax=ax)
    ax.set_title(config.bitmap[i][0])
    ax.tick_params(axis='both', which='both', length=0)
    ax.axis('off')
    # break

# Hide empty subplots if there are any
for i in range(num_plots, len(axs)):
    axs[i].axis("off")

# Adjust the layout and padding between subplots
plt.tight_layout()

# Show the plot
plt.show()

In [ ]:
FISHbasis = TMG.Layers[0].adata.X

In [ ]:
FISHbasis.mean(0)

In [ ]:
""" Scale Bits to same scale """
correction = FISHbasis.mean()/FISHbasis.mean(0)
correction

In [ ]:
(FISHbasis*correction).mean(0)

In [ ]:
FISHbasis_scaled = FISHbasis*correction

In [ ]:
data = TMG.Layers[0].adata
# Create a 5x5 grid of subplots
fig, axs = plt.subplots(5, 5, figsize=(25, 25))
x = data.obs['stage_x']
y = data.obs['stage_y']
# Flatten the axs array so that we can iterate over it using a single loop
axs = axs.ravel()
num_plots = data.X.shape[1]
# Iterate over the subplots and plot each scatter plot
for i in range(num_plots):
    c = FISHbasis_scaled[:,i].copy()
    vmin,vmax = np.percentile(c[c!=0],[5,95])
    ax = axs[i]
    scatter_plot = ax.scatter(x, y,c=c,vmin=vmin,vmax=vmax,s=0.1,marker='x',cmap='jet')  # Adjust 's' for marker size if needed
    fig.colorbar(scatter_plot, ax=ax)
    ax.set_title(config.bitmap[i][0])
    ax.tick_params(axis='both', which='both', length=0)
    ax.axis('off')
    # break

# Hide empty subplots if there are any
for i in range(num_plots, len(axs)):
    axs[i].axis("off")

# Adjust the layout and padding between subplots
plt.tight_layout()

# Show the plot
plt.show()

In [ ]:
""" Cell Staining Correction """
np.sum(FISHbasis_scaled,axis=1)

In [ ]:
np.sum(FISHbasis_scaled,axis=1).mean()

In [ ]:
correction = np.sum(FISHbasis_scaled,axis=1).mean()/np.sum(FISHbasis_scaled,axis=1)
correction

In [ ]:
FISHbasis_normalized = (FISHbasis_scaled.T*correction).T
FISHbasis_normalized

In [ ]:
data = TMG.Layers[0].adata
# Create a 5x5 grid of subplots
fig, axs = plt.subplots(5, 5, figsize=(25, 25))
x = data.obs['stage_x']
y = data.obs['stage_y']
# Flatten the axs array so that we can iterate over it using a single loop
axs = axs.ravel()
num_plots = data.X.shape[1]
# Iterate over the subplots and plot each scatter plot
for i in range(num_plots):
    c = FISHbasis_normalized[:,i].copy()
    vmin,vmax = np.percentile(c[c!=0],[5,95])
    ax = axs[i]
    scatter_plot = ax.scatter(x, y,c=c,vmin=vmin,vmax=vmax,s=0.1,marker='x',cmap='jet')  # Adjust 's' for marker size if needed
    fig.colorbar(scatter_plot, ax=ax)
    ax.set_title(config.bitmap[i][0])
    ax.tick_params(axis='both', which='both', length=0)
    ax.axis('off')
    # break

# Hide empty subplots if there are any
for i in range(num_plots, len(axs)):
    axs[i].axis("off")

# Adjust the layout and padding between subplots
plt.tight_layout()

# Show the plot
plt.show()

In [ ]:
""" Zscore """
FISHbasis_zscored = np.zeros_like(FISHbasis_normalized)
for i in range(FISHbasis_normalized.shape[1]):
    c = FISHbasis_normalized[:,i]
    vmin,vmid,vmax = np.percentile(c[np.isnan(c)==False],[25,50,75])
    c = c-vmid
    c = c/(vmax-vmin)
    FISHbasis_zscored[:,i] = c
FISHbasis_zscored

In [ ]:
data = TMG.Layers[0].adata
# Create a 5x5 grid of subplots
fig, axs = plt.subplots(5, 5, figsize=(25, 25))
x = data.obs['stage_x']
y = data.obs['stage_y']
# Flatten the axs array so that we can iterate over it using a single loop
axs = axs.ravel()
num_plots = data.X.shape[1]
# Iterate over the subplots and plot each scatter plot
for i in range(num_plots):
    c = FISHbasis_zscored[:,i].copy()
    vmin,vmax = np.percentile(c[c!=0],[5,95])
    ax = axs[i]
    scatter_plot = ax.scatter(x, y,c=c,vmin=vmin,vmax=vmax,s=0.1,marker='x',cmap='jet')  # Adjust 's' for marker size if needed
    fig.colorbar(scatter_plot, ax=ax)
    ax.set_title(config.bitmap[i][0])
    ax.tick_params(axis='both', which='both', length=0)
    ax.axis('off')
    # break

# Hide empty subplots if there are any
for i in range(num_plots, len(axs)):
    axs[i].axis("off")

# Adjust the layout and padding between subplots
plt.tight_layout()

# Show the plot
plt.show()

In [ ]:
TMG.Layers[0].adata.X = FISHbasis_zscored

In [ ]:
data = TMG.Layers[0].adata
# Create a 5x5 grid of subplots
fig, axs = plt.subplots(5, 5, figsize=(25, 25))
x = data.obs['stage_x']
y = data.obs['stage_y']
# Flatten the axs array so that we can iterate over it using a single loop
axs = axs.ravel()
num_plots = data.X.shape[1]
# Iterate over the subplots and plot each scatter plot
for i in range(num_plots):
    c = data.X[:,i].copy()
    vmin,vmax = np.percentile(c[c!=0],[5,95])
    ax = axs[i]
    scatter_plot = ax.scatter(x, y,c=c,vmin=vmin,vmax=vmax,s=0.1,marker='x',cmap='jet')  # Adjust 's' for marker size if needed
    fig.colorbar(scatter_plot, ax=ax)
    ax.set_title(config.bitmap[i][0])
    ax.tick_params(axis='both', which='both', length=0)
    ax.axis('off')
    # break

# Hide empty subplots if there are any
for i in range(num_plots, len(axs)):
    axs[i].axis("off")

# Adjust the layout and padding between subplots
plt.tight_layout()

# Show the plot
plt.show()

In [ ]:
import scanpy as sc
import umap
adata = TMG.Layers[0].adata.copy()
# adata.X = FISHbasis_norm
sc.pp.neighbors(adata, n_neighbors=15, use_rep='X') 
sc.tl.umap(adata, min_dist=0.1)
sc.tl.louvain(adata)
sc.pl.umap(adata, color='louvain', show=True)
xy = np.stack([adata.obs['stage_x'],adata.obs['stage_y']])
adata.obsm['stage'] = xy.T
sc.pl.embedding(adata, basis='stage', color='louvain', show=False)

In [ ]:
# Create a 5x5 grid of subplots
fig, axs = plt.subplots(5, 5, figsize=(25, 25))
x = data.obs['stage_x']
y = data.obs['stage_y']
# Flatten the axs array so that we can iterate over it using a single loop
axs = axs.ravel()
num_plots = len(adata.obs['louvain'].unique())
# Iterate over the subplots and plot each scatter plot
for i in adata.obs['louvain'].unique():
    i = int(i)
    # c = FISHbasis_norm[:,i].copy()
    # vmin,vmax = np.percentile(c[c!=0],[5,95])
    ax = axs[i]
    m = adata.obs['louvain']==str(i)
    scatter_plot = ax.scatter(x[m], y[m],c=adata.uns['louvain_colors'][i],s=0.1,marker='x')  # Adjust 's' for marker size if needed
    # fig.colorbar(scatter_plot, ax=ax)
    ax.set_title(str(i))
    ax.tick_params(axis='both', which='both', length=0)
    ax.axis('off')
    # break

# Hide empty subplots if there are any
for i in range(num_plots, len(axs)):
    axs[i].axis("off")

# Adjust the layout and padding between subplots
plt.tight_layout()

# Show the plot
plt.show()

In [ ]:
# SLOW
TMG.Layers[0].build_feature_graph(metric='correlation')

In [ ]:
TMG.Layers[0].adata

In [ ]:
TMG.Layers[0].adata.obsp['FG']

In [ ]:
TMG.Layers[0].adata.obsp['SG']

In [ ]:
# Create the classifier
optleiden = Classification.OptimalLeidenUnsupervized(TMG.Layers[0])

In [ ]:
# train the classifier
optleiden.train(opt_params={'iters':10, 'n_consensus':1})

In [ ]:
# use the classifier to create types and add them to TMG using the Taxonomy created on the fly by the classifier
type_vec = optleiden.classify(TMG.Layers[0].feature_mat)


In [ ]:
TMG.add_type_information(0, type_vec, optleiden.tax)

In [ ]:
TMG.create_isozone_layer()
logging.info(f"TMG has {len(TMG.Layers)} Layers")

In [ ]:
n_topics_list = [2,5,10,15,20,30,50]
n_procs = len(n_topics_list) 

topic_cls = Classification.TopicClassifier(TMG.Layers[0])
topic_cls.train(n_topics_list=n_topics_list, n_procs=n_procs)
topics = topic_cls.classify(topic_cls.Env)

In [ ]:
TMG.create_region_layer(topics, topic_cls.tax)
logging.info(f"TMG has {len(TMG.Layers)} Layers")

In [ ]:
TMG.add_geoms(geom_types = ["mask","voronoi","isozones","regions"],redo=False)

In [ ]:
TMG.save()

In [ ]:
TMG = TissueMultiGraph(inputpath=inputpath,redo = False)

In [ ]:
V = BasisView(TMG,rotation=-90)
V.show()

In [ ]:
SingleMapView(TMG,level_type = "cell", map_type = "random",rotation=-90).show()
SingleMapView(TMG,level_type = "isozone",map_type = "random",rotation=-90).show()
SingleMapView(TMG,level_type = "region",map_type = "random",rotation=-90).show()


In [ ]:
colormaps = ['Purples','Oranges','Blues','Greens','Reds','cividis']
SingleMapView(TMG,level_type = "cell", map_type = "type",color_assign_method = 'linkage',colormaps = colormaps,rotation=-90).show()


In [ ]:
V = UMAPwithSpatialMap(TMG,qntl = (0.025,0.975),clp_embed = (0.025,0.975),rotation=-90)
V.show()